# TP 5 : Plannification d'examen - Correction

In [1]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

Dans une école d'ingénieurs de la région parisienne, chaque semestre, chaque étudiant de deuxième année 
choisit un ensemble de huit modules parmi onze proposés, en fonction de l'option qu'il désire suivre en troisième année. Ces options sont au nombre de deux : *Aide à la décision et optimisation* et *Communication homme-machine et documents électroniques*.

Pour le semestre courant, certains modules sont obligatoires pour les étudiants désirant 
s'orienter vers ces deux options. Il s'agit des modules de Probabilités (P), Algorithme et complexité III (AC), Ingénierie des connaissances (IC) et Systèmes distribués et parallélisme (SDP). Certains autres sont optionnels : Bases de données II (BD), Réseaux (R), Optimisation combinatoire II (OC), Interface graphique (IG), C++ (C++), Paradigme logique (PL) et Génie logiciel III (GL).

&nbsp;       |BD      |R       |C++     |GL      |IC      |IG      |AC      |PL      |OC      |P       |SDP
-------------|------- |------- |------- |------- |------- |------- |------- |------- |------- |------- |----
BD           | &nbsp; | x      | &nbsp; | &nbsp; | x      | &nbsp; | x      | &nbsp; | &nbsp; | x      | x
R            | x      | &nbsp; | &nbsp; | &nbsp; | x      | &nbsp; | x      | &nbsp; | &nbsp; | x      | x
C++          | &nbsp; | &nbsp; | &nbsp; | x      | x      | x      | x      | &nbsp; | x      | x      | x
GL           | &nbsp; | &nbsp; | x      | &nbsp; | x      | x      | x      | &nbsp; | &nbsp; | x      | x
IC           | x      | x      | x      | x      | &nbsp; | x      | x      | x      | x      | x      | x
IG           | &nbsp; | &nbsp; | x      | x      | x      | &nbsp; | x      | &nbsp; | x      | x      | x
AC           | x      | x      | x      | x      | x      | x      | &nbsp; | x      | x      | x      | x
PL           | &nbsp; | &nbsp; | &nbsp; | &nbsp; | x      | &nbsp; | x      | &nbsp; | &nbsp; | x      | x
OC           | &nbsp; | &nbsp; | x      | &nbsp; | x      | x      | x      | &nbsp; | &nbsp; | x      | x
P            | x      | x      | x      | x      | x      | x      | x      | x      | x      | &nbsp; | x
SDP          | x      | x      | x      | x      | x      | x      | x      | x      | x      | x      |&nbsp;

M. Eudété doit planifier les examens prévus à la fin du semestre. Chaque examen dure deux heures.
Deux journées sont réservées pour planifier ces examens suivant les plages horaires suivantes: 
8h-10h, 10h15-12h15, 14h-16h et 16h15-18h15. Pour chaque examen, elle connaît l'ensemble des examens incompatibles, 
qui ne peuvent avoir lieu en même temps car devant être effectués par des étudiants communs. 
Ces incompatibilités sont résumées dans le tableau, une croix indiquant une incompatibilité.

On souhaite que la distribution des examens sur les deux journées soit la plus équilibrée possible;
néanmoins, le nombre d'examens planifiés pour la deuxième journée ne doit pas être supérieur au nombre d'examens planifiés pour la première journée.


## Question 1

Aidez M. Eudété à construire un emploi du temps de telle sorte qu'aucun étudiant n'ait plus d'un examen en même temps,
en proposant (sur papier) une formulation PLNE pour résoudre le problème.

*Suggestion :* utiliser une famille de variables $x$ pour represénter les décisions qui concernent l'affectation des examens aux créneaux horaires,
et une variable $z$ pour modéliser à la fois la fonction objectif et les contraintes sur le nombre et la distribution des examens sur les deux journées.

**CORRECTION :**


Notons:
* $n$, le nombre d'examens, et $E = \{0, \dots, n-1\}$, l'ensemble des examens
* $C_E = \{(e_1,e_2):e_1<e_2\}$ l'ensemble des couples $(e_1,e_2)$ d'examens, et $A_E \subset C_E$, l'ensemble des couples d'examens incompatibles.
  Étant donné un couple d'examens incompatibles $c\in A_E$, on indiquera les examens qui le composent comme $e_1(c)$ et $e_2(c)$
* $H_1 = \{0,\dots, 3\}$, l'ensemble des créneaux horaires du premier jour, $H_2 = \{4, \dots, 7\}$, l'ensemble des créneaux horaires du deuxième jour, et
	$H = H_1\cup H_2$, l'ensemble de tous les créneaux.


On utilise les variables décisionnelles suivantes:
* $x_{e,h}\in\{0,1\},\quad e\in E,h\in H,\quad x_{e,h} = 1 \Leftrightarrow$ le créneau choisi pour l'examen $e$ est $h$
* $z\in \{0, \dots, n\}$ le nombre d'examens supplémentaires le premier jour par rapport au deuxième jour.


**CORRECTION :**


$\min z$	

$z=\displaystyle\sum_{\substack{e\in E\\h\in H_1}}x_{e,h} -\displaystyle\sum_{\substack{e\in E\\h\in H_2}}x_{e,h}$

$\displaystyle\sum_{h\in H}x_{e,h} = 1 \quad\quad \forall e\in E$

$x_{e_1(c),h} + x_{e_2(c),h}\leq 1 \quad\quad \forall c\in A_E,\forall h\in H$

$x_{e,h} \in \{0,1\} \quad\quad \forall e \in E, \forall h\in H$

$z \geq 0$


## Question 2

M. Eudété ne sait pas comment résoudre le PLNE. Aidez-le en le résolvant pour lui avec Julia et JuMP.

*Remarque :* comme M. Eudété a besoin de le faire tous les ans, proposez une formulation générique pour qu'il n'ait qu'à changer les données l'année prochaine.


In [2]:
matières = ["BD", "R", "C++", "GL", "IC", "IG", "AC", "PL", "OC", "P", "SDP"]

incompatibilités = [
0 1 0 0 1 0 1 0 0 1 1 ;
1 0 0 0 1 0 1 0 0 1 1 ;
0 0 0 1 1 1 1 0 1 1 1 ;
0 0 1 0 1 1 1 0 0 1 1 ;
1 1 1 1 0 1 1 1 1 1 1 ;
0 0 1 1 1 0 1 0 1 1 1 ;
1 1 1 1 1 1 0 1 1 1 1 ;
0 0 0 0 1 0 1 0 0 1 1 ;
0 0 1 0 1 1 1 0 0 1 1 ;
1 1 1 1 1 1 1 1 1 0 1 ;
1 1 1 1 1 1 1 1 1 1 0 
]

horaires = ["8h-10h", "10h15-12h15", "14h-16h", "16h15-18h15"]

H = 1:length(horaires) # 4
for horaires ∈ H
    print(horaires) # 1234
end
print(horaires ∈ H)



1234false

## Question 3

M. Eudété ne sait pas lire les solutions données sous forme d'une solution 0/1. Affichez la solution de manière lisible en indiquant pour chaque matière le créneau horaire où se déroule l'examen. Affichez également le nombre d'examens en plus le premier jour.

In [3]:
##################
#   Correction   #
##################


function optimise_edt(m, i, h)

    n = length(m)
    M = 1:n # nombre de matiere
    
    t = length(h)
    H = 1:t # nombre de horaire
    
    # On a deux jours
    J = 1:2 
    
    edt = Model(GLPK.Optimizer)

    #variables y[p] dans 0,1 : est-ce que j'utilise le rouleau p pour p=1,...P
    @variable(edt, z >= 0) # day 1 - day 2 >= 0

    #varbiale x[m, j, h] = 1 si le contrôle de la matière m a lieu le jour j à l'heure h, 0 sinon
    @variable(edt, x[M, J, H], Bin) 

    #La fonction objectif : On cherche à minimiser le nombre de rouleaux decoupes.
    @objective(edt, Min, z)

    
    @constraint(edt, sum(x[m, 1, h] for m ∈ M, h ∈ H) - sum(x[m, 2, h] for m ∈ M, h ∈ H) == z)

    for m ∈ M
        @constraint(edt, sum(x[m,j,h] for j ∈ J, h ∈ H) == 1)
    end
    
    
    for m ∈ M, m2 ∈ M
        if m != m2 && i[m, m2] == 1 # 2 matiere different with incomtabibite = 1
            for h ∈ H, j ∈ J
                @constraint(edt, x[m,j,h] + x[m2, j, h] <= 1)
            end
        end
    end
    
    #print(edt)
    JuMP.optimize!(edt)

    for m in M
        for j ∈ J, h ∈ H
            if JuMP.value(x[m,j,h]) != 0
                println(matières[m], " aura lieu le jour $j à l'horaire ", horaires[h])
            end
        end
    end
end



optimise_edt (generic function with 1 method)

In [4]:
optimise_edt(matières, incompatibilités, horaires)